# TFX - Compile and Deploy the TFX Pipeline to KFP

This Notebook helps you to compile the **TFX Pipeline** to a **KFP package**. This will creat an **Argo YAML** file in a **.tar.gz** package. We perform the following steps:
1. Build a custom container image that include our modules
2. Compile TFX Pipeline using CLI
3. Deploy the compiled pipeline to KFP 


## Setup

In [ ]:
!pip install -U kfp

## Set compile time variables

In [ ]:
import os

os.environ['PROJECT_ID'] = 'ksalama-cloudml' # Set your project
os.environ['BUCKET'] = 'ksalama-cloudml' # Set your project
os.environ['REGION'] = 'us-central1' # Set your region
os.environ['GKE_CLUSTER_NAME'] = 'ksalama-cloudml-dev' # Set your GKE cluster name
os.environ['GKE_CLUSTER_ZONE'] = 'europe-west2-a' # Set your GKE cluster zone


os.environ['IMAGE_NAME'] = 'tfx-ml'
os.environ['TAG'] = 'latest'
os.environ['ML_IMAGE_URI']=f'gcr.io/{os.environ.get("PROJECT_ID")}/{os.environ.get("IMAGE_NAME")}:{os.environ.get("TAG")}'

os.environ['NAMESPACE'] = 'kubeflow-pipelines'
os.environ['ARTIFACT_STORE_URI'] = f'gs://{os.environ.get("BUCKET")}/tfx_artifact_store'
os.environ['GCS_STAGING_PATH'] = f'{os.environ.get("ARTIFACT_STORE_URI")}/staging'

os.environ['RUNTIME_VERSION'] = '2.2'
os.environ['PYTHON_VERSION'] = '3.7'
os.environ['BEAM_RUNNER'] = 'DirectRunner'
os.environ['MODEL_REGISTRY_URI'] = f'{os.environ.get("ARTIFACT_STORE_URI")}/model_registry'

os.environ['PIPELINE_NAME'] = 'tfx_bqml_scann_embeddings'

## 1. Build Container Image

The pipeline uses a custom docker image, which is a derivative of the [tensorflow/tfx:0.23.0](https://hub.docker.com/r/tensorflow/tfx) image, as a runtime execution environment for the pipeline's components. The same image is also used as a a training image used by **AI Platform Training**.


In [ ]:
!gcloud builds submit --tag $ML_IMAGE_URI tfx_pipeline

## 2. Compile TFX Pipeline using CLI

In [ ]:
!rm ${PIPELINE_NAME}.tar.gz
!tfx pipeline compile \
    --engine=kubeflow \
    --pipeline_path=tfx_pipeline/runner.py 

## 3. Deploy the Compiled Pipeline to KFP

In [ ]:
%%bash

gcloud container clusters get-credentials ${GKE_CLUSTER_NAME} --zone ${GKE_CLUSTER_ZONE}
export KFP_ENDPOINT=$(kubectl describe configmap inverse-proxy-config -n ${NAMESPACE} | grep "googleusercontent.com")

kfp --namespace=${NAMESPACE} --endpoint=${KFP_ENDPOINT} \
    pipeline upload \
    --pipeline-name=${PIPELINE_NAME} \
    ${PIPELINE_NAME}.tar.gz

## Use the KFP UI to run the deployed pipeline...

In [ ]:
!echo $(kubectl describe configmap inverse-proxy-config -n ${NAMESPACE} | grep "googleusercontent.com")

## License

Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at: http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 

See the License for the specific language governing permissions and limitations under the License.

**This is not an official Google product but sample code provided for an educational purpose**